In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su única responsabilidad es crear la tabla de la Capa de Bronce (Bronze).
import re
from pathlib import Path

# 1. --- Configuración de la Capa de Bronce ---
# La ruta al archivo crudo que subiste a tu "Volume"
source_volume_path = "/Volumes/workspace/tecnomundo_data_raw/uploads_raw/Reporte de ventas por articulos-2.csv"

# Definimos explícitamente el catálogo y el schema de la Capa de Bronce
bronze_catalog = "workspace"
bronze_schema = "tecnomundo_data_raw"

print(f"Ruta de Origen (Volume): {source_volume_path}")
print(f"Schema de Destino (Bronce): {bronze_catalog}.{bronze_schema}")


# 2. --- Generación Dinámica del Nombre de la Tabla de Bronce ---
# Se toma el nombre del archivo de la ruta de origen.
file_name = Path(source_volume_path).name

# Se limpia el nombre para que sea un nombre de tabla SQL válido.
table_base_name = Path(source_volume_path).stem
sanitized_name = re.sub(r'[\s\.\-]+', '_', table_base_name).lower()
sanitized_name = re.sub(r'[^a-zA-Z0-9_]', '', sanitized_name)

# Se construye el nombre final de la tabla de Bronce.
bronze_table_name = f"{sanitized_name}_raw"
full_bronze_table_name = f"{bronze_catalog}.{bronze_schema}.{bronze_table_name}"

print(f"El archivo de origen es: {file_name}")
print(f"El nombre de la tabla de destino (Bronce) será: {full_bronze_table_name}")


# 3. --- Proceso de Ingesta a la Capa de Bronce ---
try:
  # --- Limpieza Previa (Opcional pero recomendado) ---
  # Se elimina la tabla si existe para asegurar que se cree desde cero.
  print(f"\nAsegurando que la tabla de destino '{full_bronze_table_name}' no exista...")
  spark.sql(f"DROP TABLE IF EXISTS {full_bronze_table_name}")
  print("Limpieza previa completada.")

  print(f"\nIniciando la lectura del archivo: {source_volume_path}")
  
  df_raw = (spark.read
            .format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .load(source_volume_path))
  
  print("Lectura del archivo completada exitosamente.")

  # 4. --- Saneamiento de Nombres de Columnas ---
  # Limpiamos los nombres de las columnas ANTES de guardar para asegurar la compatibilidad.
  print("\nSaneando nombres de columnas...")
  df_sanitized = df_raw
  for col in df_sanitized.columns:
      new_col_name = col.replace('º', '').replace('Nº', 'num').replace('ñ', 'n').replace('Ñ', 'N')
      new_col_name = re.sub(r'[\s\.\-]+', '_', new_col_name)
      new_col_name = re.sub(r'[^a-zA-Z0-9_]', '', new_col_name).lower()
      
      if new_col_name != col:
          df_sanitized = df_sanitized.withColumnRenamed(col, new_col_name)
          print(f"  - Columna renombrada: '{col}' -> '{new_col_name}'")
  
  # 5. --- Guardado en la Capa de Bronce ---
  # Ahora guardamos el DataFrame con los nombres de columna ya limpios.
  print(f"\nIniciando el guardado en la tabla de Bronce: {full_bronze_table_name}")
  
  (df_sanitized.write
   .option("overwriteSchema", "true") # Permite que el esquema de la tabla cambie si el origen cambia.
   .mode("overwrite")
   .saveAsTable(full_bronze_table_name))
  
  print(f"¡Éxito! Los datos han sido ingestados y guardados en la tabla de Bronce '{full_bronze_table_name}'.")
  
  # 6. --- Verificación ---
  print("\nMostrando una muestra de los datos cargados en la Capa de Bronce:")
  final_df = spark.table(full_bronze_table_name)
  display(final_df)
  
  print("\nEsquema de la tabla de Bronce (nombres de columna saneados):")
  final_df.printSchema()

except Exception as e:
  print(f"Ocurrió un error durante el proceso de ingesta: {e}")
